# Synpsis

Use case: Import source text and save in F3 form.

# Configuration

In [1]:
body_start = 341
body_end = 21964
chap_pat = r'^\s*(?:CHAPTER|ETYMOLOGY|Epilogue).*$'
para_pat = r'\n\n+'
sent_pat = r'([.;?!"“”]+)'
token_pat = r'([\W_]+)'
db_file = 'moby.db'
src_file_name = '2701-0.txt'
src_file_url = 'https://www.gutenberg.org/files/2701/2701-0.txt'

In [2]:
extra_stopwords = """
us rest went least would much must long one like much say well without though yet might still upon
done every rather particular made many previous always never thy thou go first oh thee ere ye came
almost could may sometimes seem called among another also however nevertheless even way one two three
ever put
""".strip().split()

In [3]:
set(extra_stopwords)

{'almost',
 'also',
 'always',
 'among',
 'another',
 'called',
 'came',
 'could',
 'done',
 'ere',
 'even',
 'ever',
 'every',
 'first',
 'go',
 'however',
 'least',
 'like',
 'long',
 'made',
 'many',
 'may',
 'might',
 'much',
 'must',
 'never',
 'nevertheless',
 'oh',
 'one',
 'particular',
 'previous',
 'put',
 'rather',
 'rest',
 'say',
 'seem',
 'sometimes',
 'still',
 'thee',
 'thou',
 'though',
 'three',
 'thy',
 'two',
 'upon',
 'us',
 'way',
 'well',
 'went',
 'without',
 'would',
 'ye',
 'yet'}

In [4]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']
CHAPS = OHCO[:1]
PARAS = OHCO[:2]
SENTS = OHCO[:3]

# Libraries

In [5]:
import re
import os
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/leonardramsey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/leonardramsey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leonardramsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/leonardramsey/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leonardramsey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Pragmas

In [6]:
%matplotlib inline

# Process

We pause to look at the revised form of our text import function. The parsing function has been replaced with NLTK, which has improved the results of POS tagging. However, this has required some added string manipulation to produce better tokens.

## Download if necessary

In [7]:
if not os.path.exists(src_file_name):
  import requests
  with open(src_file_name, 'w', encoding='utf-8') as src_file_on_disk:
    src_file_url = src_file_url
    src_file_text = requests.get(src_file_url).text
    src_file_on_disk.write(src_file_text)

## Text to lines

In [8]:
lines = open(src_file_name, 'r', encoding='utf-8').readlines()
lines = lines[body_start - 1 : body_end + 1]
df = pd.DataFrame({'line_str':lines})
df.index.name = 'line_id'
del(lines)

## Fix some characters to improve tokenization

In [9]:
df.line_str = df.line_str.str.replace('—', ' — ')
df.line_str = df.line_str.str.replace('-', ' - ')

## Lines to Chapters

In [10]:
chap_mask = df.line_str.str.match(chap_pat)
df.loc[chap_mask, 'chap_id'] = df.apply(lambda x: x.name, 1)

In [11]:
df.chap_id = df.chap_id.ffill().astype('int')
chap_ids = df.chap_id.unique().tolist()
df['chap_num'] = df.chap_id.apply(lambda x: chap_ids.index(x))
chaps = df.groupby('chap_num')\
    .apply(lambda x: ''.join(x.line_str))\
    .to_frame()\
    .rename(columns={0:'chap_str'})
del(df)

## Chapters to Paragraphs

In [12]:
paras = chaps.chap_str.str.split(para_pat, expand=True)\
    .stack()\
    .to_frame()\
    .rename(columns={0:'para_str'})
paras.index.names = PARAS
paras.para_str = paras.para_str.str.strip()
paras.para_str = paras.para_str.str.replace(r'\n', ' ')
paras.para_str = paras.para_str.str.replace(r'\s+', ' ')
paras = paras[~paras.para_str.str.match(r'^\s*$')]
del(chaps)

## Paragraphs to Sentences

In [13]:
#     sents = paras.para_str.str.split(sent_pat, expand=True)\
sents = paras.para_str\
    .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
    .stack()\
    .to_frame()\
    .rename(columns={0:'sent_str'})
sents.index.names = SENTS
del(paras)

## Sentences to Tokens with POS tagging

In [14]:
#     tokens = sents.sent_str.str.split(token_pat, expand=True)\
tokens = sents.sent_str\
    .apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))\
    .stack()\
    .to_frame()\
    .rename(columns={0:'pos_tuple'})
tokens.index.names = OHCO
tokens['pos'] = tokens.pos_tuple.apply(lambda x: x[1])
tokens['token_str'] = tokens.pos_tuple.apply(lambda x: x[0])
tokens = tokens.drop('pos_tuple', 1)
del(sents)

## Tag punctuation and numbers

In [15]:
tokens['punc'] = tokens.token_str.str.match(r'^[\W_]*$').astype('int')
tokens['num'] = tokens.token_str.str.match(r'^.*\d.*$').astype('int')

## Extract vocab with minimal normalization

In [16]:
WORDS = (tokens.punc == 0) & (tokens.num == 0)
tokens.loc[WORDS, 'term_str'] = tokens.token_str.str.lower()\
    .str.replace(r'["_*.]', '')
vocab = tokens[tokens.punc == 0].term_str.value_counts().to_frame()\
    .reset_index()\
    .rename(columns={'index':'term_str', 'term_str':'n'})
vocab = vocab.sort_values('term_str').reset_index(drop=True)
vocab.index.name = 'term_id'

## Get priors for Vocab

In [17]:
vocab['p'] = vocab.n / vocab.n.sum()

## Add stems

In [18]:
stemmer = nltk.stem.porter.PorterStemmer()
vocab['port_stem'] = vocab.term_str.apply(lambda x: stemmer.stem(x))

## Define stopwords

In [19]:
stopwords = set(nltk.corpus.stopwords.words('english') + extra_stopwords)

In [20]:
# stopwords

In [21]:
sw = pd.DataFrame({'x':1}, index=stopwords)
vocab['stop'] = vocab.term_str.map(sw.x).fillna(0).astype('int')
del(sw)

## Add term_ids to Tokens 

In [22]:
tokens['term_id'] = tokens['term_str'].map(vocab.reset_index()\
    .set_index('term_str').term_id).fillna(-1).astype('int')

# Save

In [23]:
with sqlite3.connect(db_file) as db:
    tokens.to_sql('token', db, if_exists='replace', index=True)
    vocab.to_sql('vocab', db, if_exists='replace', index=True)

In [24]:
# END